# <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png"> CS207 Systems Development for Computational Science
## Final Project
## Milestone 2 



**Harvard University**<br/>
**Fall 2019**<br/>
**Instructor**: David Sondak <br/>
**Team #14**: Fantastic Four<br/>
**Students**: Daniel Cox, Anna Davydova, Stephen Moon, Valentina Toll Villagra   <br/>
**Git Repository**:https://github.com/IACS-CS-207-FantasticFour/cs207-FinalProject


<hr style="height:2pt">

*“Nothing takes place in the world whose meaning is not that of some maximum or minimum.”*
― Leonhard Euler

**INTRODUCTION:**<br/>
<br/>
This project aims to deliver an Automatic Differentiation Software DeltaPi that will calculate derivatives efficiently and accurately, making it useful across a wide spectrum of applications. In this package, we will execute the forward mode of Automatic Differentiation.  We will also provide an extension package HedgeDeltaPi that will calculate delta for European Call and Put options, allowing for efficient delta hedging. We are especially excited about the applications of this software on Wall Street, where Automatic Differentiation has been adopted fairly recently and has already shown significant improvements in efficiency and/or accuracy vs. the alternative methods (Hedin, 2019) . <br/>
<br/>
Derivatives are powerful and ubiquitous. Their use spans from gradients and Hessians in machine learning applications to hedge sensitivities in financial markets. However, of the four existing methods for derivative calculation only Automatic  Differentiation (AD) combines interpretability, efficiency and accuracy. Manual differentiation is inefficient and susceptible to error.(Baydin et. al., 2018). Numerical differentiation, while easy and fairly quick in its implementation, is also inaccurate, prone to rounding and truncation errors (Jerrell, 1997).  It does not scale well, which makes it a poor choice for machine learning models. Symbolic differentiation, while accurate, can become incredibly complex facing "expression swell" issues (Corliss, 1988). AD overcomes these issues, via its application of the chain rule and a step by step approach to differentiation, and accurately computes derivatives with asymptotic efficiency. (Baydin et.al, 2018). <br/>
<br/>
Our software DeltaPi will implement AD methods, allowing the end user to benefit from its accuracy and efficiency.  Our goal is to produce a package that can handle a wide variety of uses beyond simple scalar functions and an extension package to compute deltas for financial options on Wall Street.

**BACKGROUND:** <br/>
<br/>
Generally speaking, in any computer program a function can be broken down into its elementary function components (unary or binary) such as addition, subtraction  log, sin, sqrt etc (Heath, 2018).  Since the value of partial derivatives of these elementary functions can be easily calculated, then the value of the entire function can be calculated via the application of the chain rule.<br/>

Let us walk through this process in more detail. Fundamentally, Chain Rule is the key pillar to the AD process. Recall that:

$$\dfrac{\partial h}{\partial t} = \dfrac{\partial h}{\partial u}\dfrac{\partial u}{\partial t}$$ 

We can extend the definition of the Chain Rule to a function that contains to functions as follows: h(u(t),v(t)) (Sondak, 2019).:

\begin{align}
  \displaystyle 
  \frac{\partial h}{\partial t} = \frac{\partial h}{\partial u}\frac{\partial u}{\partial t} + \frac{\partial h}{\partial v}\frac{\partial v}{\partial t}.
\end{align}

This result, from a perspetive of a gradient, leads us to this general rule:

\begin{align}
  \nabla_{x}h = \sum_{i=1}^{n}{\frac{\partial h}{\partial y_{i}}\nabla y_{i}\left(x\right)}.
\end{align}

Thus, in a nutshell, the AD process consist of breaking down the function into its elementary components and carrying out the differentiation process in sequential order of these operations while multiplying through with the chain rule. The output of this process is a dual number containing the value of the function along with its derivative.  We can visualize this process with a computational graph and a table that contains the trace of the calculations. 

For example, for a simple function (*adapted from class exercises*): $$f\left(x,y\right) = (sin(x)-cos(y))^2.$$ 

The forward mode AD graph looks as follows:

<img src="simple_example.png" width="500" height="240" align="center"/>

The corresponding computational table for values $x=\frac{\pi}{2}$ and $y=\frac{\pi}{3}$looks as follows:

| Trace    | Elementary Operation &nbsp;&nbsp;&nbsp;| Current Function Value &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;     | Derivative  &nbsp;&nbsp;&nbsp;&nbsp;   | Partial Derivative w.r.t. x                            | Partial Derivative w.r.t. y                       |evaluate f(x) and its derivative: (f(x), f'(x))                   |
| :------: | :-------------------------------------:| :--------------------------------------------------------:|:-------------------------------------: | :-------------------------------------------------:|:----------------------------------------------:|:---------------------------------------:
| $x_{1}$  | $x_{1}$                                |          $\frac{\pi}{2}$                                              |$\dot {x_1}$                                     |      $1$                                           |      $0$                                       |$\left(\frac{\pi}{2}, 1\right)$                                                                  |
| $x_{2}$  | $x_{2}$                                |          $\frac{\pi}{3}$                                              |$\dot {x_2}$                                     |      $0$                                           |      $1$                                       |$\left(\frac{\pi}{3},1 \right)$                                                                  |
| $x_{3}$  | $sin(x_1)$                             |          $1$                                          |$cos(x_1)\dot{x_1}$                          |      $0$                                          |      $0$                                       |$\left(1,0)\right)$                                                        |
| $x_{4}$  | $cos(x_2)$                         |               $\frac{1}{2}$                                            |$-sin(x_2)\dot{x_2}$                     |      $0$                                          |    $-\frac{\sqrt{3}}{2}$                             |$\left(\frac{1}{2},-\frac{\sqrt{3}}{2}\right )$                                                         |
| $x_{5}$  | $x_3-x_4$                          |          $\frac{1}{2}$                                        |$\dot{x_3}-\dot{x_5}$           |     $0$                                        |   $\frac{\sqrt{3}}{2}$                            |$\left(\frac{1}{2},\frac{\sqrt{3}}{2}\right)$                                                         |
| $x_{6}$  | $x_5^2$                             |    $\frac{1}{4}$                                     |  $2x_5\dot{x_5}$                         |                        $0$                            |       $\frac{\sqrt{3}}{2}$ |           $\left(\frac{1}{4},\frac{\sqrt{3}}{2}\right)$      |                                                                                                        







We note not only the simplicity of differentiation process on elementary functions carried along by the chain rule but relatively interpretable process that could be useful to our end-users.

AD process generalizes nicely to calculate a function's  Jacobian where the function:</br>
$f: \mathbb{R}^n \rightarrow \mathbb{R}^m $ has n independent (input) variables $x_i$ and m dependent (output) variables $y_j$ (Baydin et.al, 2018). Here, for each forward pass only one of of the variables $\dot{x_i}$ is initialized at $\dot{x_i}=1$, while the rest are set at 0.  Every time we evaluate the derivate for a specified value of x such as $x=a$, we are computing:</br>
$$\left.\dot{y}_j=\frac{\partial{y_j}}{\partial{x_i}}\right\vert_{x=a}, j=1, . . . , m$$

This gives us one column from a Jacobian $mxn$ matrix:</br>
    $$\left. J_f=
    \begin{bmatrix}
    \frac{\partial{y_1}}{\partial{x_1}} & . & . & . & \frac{\partial{y_1}}{\partial{x_n}} \\
    . & . &  &  & .\\
    . &   & . & & .\\
    . &  & & .& . \\
    \frac{\partial{y_m}}{\partial{x_1}} & . & . & . & \frac{\partial{y_m}}{\partial{x_n}}
    \end{bmatrix}\right\vert_{x=a}$$

When we finish all of our evaluations for each value of $x_i, . . . , x_n$ we end up with the full Jacobian matrix. What is particularly useful about AD is that it provides a more efficient/matrix free way for calculating Jacobian-vector products (Baydin et.al, 2018). Consider the following Jacobian-vector product:

$$ J_fr=
    \begin{bmatrix}
    \frac{\partial{y_1}}{\partial{x_1}} & . & . & . & \frac{\partial{y_1}}{\partial{x_n}} \\
    . & . &  &  & .\\
    . &   & . & & .\\
    . &  & &. & . \\
    \frac{\partial{y_m}}{\partial{x_1}} & . & . & . & \frac{\partial{y_m}}{\partial{x_n}}
    \end{bmatrix}
    \begin{bmatrix}
    r_1\\
    .\\
    .\\
    .\\
    r_n
    \end{bmatrix}$$

All we have to do here is initialize $\dot{x}=r$ and we will be able to compute the entire Jacobian-vector product in one forward pass. In general, AD forward mode is very efficients for $f: \mathbb{R}\rightarrow \mathbb{R}^m $ with all of the  derivatives computed in one forward pass. <br/>
</br>
***Please Note:*** AD forward mode is not always efficient! In the case where we have  $f: \mathbb{R}^n \rightarrow \mathbb{R}$  AD forward mode performs n evaluations, outputting one column at a time for 1xn Jacobian matrix. Using reverse mode here would be significantly more efficient as we would just need one pass. It is also important to keep in mind that for situations where $f: \mathbb{R^n}\rightarrow \mathbb{R}^m $ where n>>m, using reverse mode would be more efficient. For a function $f: \mathbb{R^n}\rightarrow \mathbb{R}^m $ where $ops(f)$ is the number of operations contained within the function, the time it would take AD forward mode can be approximated as $n*c*ops(f)$ while for AD reverse mode the time is $m*c*ops(f)$ where c is some constant c < 6, usually in the [2, 3] range (Griewank and Walther, 2008). For more information on AD reverse mode please see [Automatic Differentiation in Machine Learning: A Survey by Baydin et al](https://arxiv.org/pdf/1502.05767.pdf).

**HOW TO USE OUR PACKAGE**

####  Pre-requisites:

import numpy as np

--------------------------------------------------------------------------------------------------------------

​ Be x,y,z,... input variables
​ Be f(x,y,z,...) the output of function f at (x,y,z,...)
​ Be AutoDiff the class 

1) Create an instance of the AutoDiff class
x = AutoDiff(val, derv)

where:
​ val = the value of x at the desired point
​ derv = the initial value of the derivative of x
    ​ 1 for calculating the partial derivative df/dx
    ​ 0 for calculating the partial derivative df/dy,df/dz,...

2) Repeat the process of creating an instance of the AutoDiff class for each input variable
y = AutoDiff(val, derv)
z = AutoDiff(val, derv)
...

where:
​ val = the value of the instance variable (y,z,...) at the desired point
​ derv = the initial value of the derivative of the instance variable (y,z,...)
    ​ 1 for calculating the partial derivative of the instance variable (df/dy for y, df/dz for z,...)
    ​ 0 for calculating the partial derivative of another variable that is not the instance one (df/dx,df/dz for y, df/dx,df/dy for z,...)
    

3) Enter the function of interest
f = function of x,y,z,...

where function can contain the following operations:
​ +
​ -
​ * 
​ /
​ **
​ exp()
​ sqrt()
​ sin()
​ cos()
​ tan()

4) Print f.val to get the calculated value of f at the specified point (x,y,z,...)
print(f.val)

5) Print f.derv to get the calculated value of df/dx or df/dy or df/dz or ... at the specified point (x,y,z,...)
print(f.derv)


#### Example

​ For values:
x = 3
y = 4
f = x + 2 * y

​ Be the goal to calculate the partial derivatives df/dx at (x,y)=(3,4)

1) Create an instance of the AutoDiff class
x = AutoDiff(3, 1)

2) Repeat the process of creating an instance of the AutoDiff class for y
y = AutoDiff(4, 0)

3) Enter the function of interest
f = x + 2 * y

4) Print f.val to get the calculated value of f at (3,4)
print(f.val)

5) Print f.derv to get the calculated value of df/dx at (3,4)
print(f.derv)


​ Be the goal to calculate the partial derivatives df/dy at the same (x,y)=(3,4)

1) Create an instance of the AutoDiff class
x = AutoDiff(3, 0)

2) Repeat the process of creating an instance of the AutoDiff class for y
y = AutoDiff(4, 1)

3) Enter the function of interest
f = x + 2 * y

4) Print f.val to get the calculated value of f at (3,4) - will be the same from df/dx
print(f.val)

5) Print f.derv to get the calculated value of df/dy at (3,4) print(f.derv)

--------------------------------------------------------------------------------------------------------------
#### To use all_derivatives() function

​ Be x,y,z,... input variables
​ Be func(x,y,z,...) the output of function f at (x,y,z,...)
​ Be all_derivatives the function that uses AutoDiff class 

1) Create a vector containing the input variables
in_vars = [x,y,z,...]

where:
​ x,y,z,... are variable names 

2) Create a vector containing the variable values
in_vals = [val_x,val_y,val_z,...]

where:
​ val_x = the value of x at which to evaluate the function
​ val_y = the value of y at which to evaluate the function
​ val_z = the value of z at which to evaluate the function
...

3) Enter the function to be evaluated
func = function of x,y,z,...

where function can contain the following operations:
​ +
​ -
​ * 
​ /
​ **
​ exp()
​ sqrt()
​ sin()
​ cos()
​ tan()

4) Run all_derivatives()
result = allderivatives(func, in_vars, in_vals)

5) Print result.val to get the calculated value of func at (in_vars)
print(result.val)

6) Print result.derv_vals to get vector of calculated partial derivative values, one for each variable in in_vars print(result.derv_vals)

#### Example

​ For values:
x = 3
y = 4
z = 5
func = x + 2 * y - z


1) Create a vector containing the input variables
in_vars = [x,y,z]

2) Create a vector containing the variable values
in_vals = [3,4,5]

3) Enter the function to be evaluated
func =  x + 2 * y - z

4) Run all_derivatives()
result = all_derivatives(func, in_vars, in_vals)

5) Print result.val to get the calculated value of func at (in_vars)
print(result.val)

6) Print result.derv_vals to get vector of calculated partial derivative values, one for each variable in in_vars print(result.derv_vals)

--------------------------------------------------------------------------------------------------------------
#### To use multi_func_all_derivatives() function

​ Be x,y,z,... input variables
​ Be func(x,y,z,...) the output of function f at (x,y,z,...)
​ Be multi_func_all_derivatives the function that uses AutoDiff class 

1) Create a vector containing the input variables
in_vars = [x,y,z,...]

where:
​ x,y,z,... are variable names 

2) Create a vector containing the variable values
in_vals = [val_x,val_y,val_z,...]

where:
​ val_x = the value of x at which to evaluate the function
​ val_y = the value of y at which to evaluate the function
​ val_z = the value of z at which to evaluate the function
...

3) Create a vector containing the functions to be evaluated
funcs = [func_1,func_2,func_3,...]

where:
func_n = function of x,y,z,...

where func_n can contain the following operations:
​ +
​ -
​ * 
​ /
​ **
​ exp()
​ sqrt()
​ sin()
​ cos()
​ tan()

4) Run multi_func_all_derivatives()
result = multi_func_all_derivatives(funcs, in_vars, in_vals)

5) Print result.out_vals to get vector of calculated values of all funcs each at (in_vars)
print(result.out_vals)

6) Print result.out_derv_matrix to get the Hessian matrix of partial derivatives evaluated at (in_vars)
print(result.out_derv_matrix)

#### Example
​ For values:
x = 3
y = 4
z = 5
func_1 = x + 2 * y - z
func_2 = x - exp(y) + z

1) Create a vector containing the input variables
in_vars = [x,y,z]


2) Create a vector containing the variable values
in_vals = [3,4,5]

3) Create a vector containing the functions to be evaluated
func_1 = x + 2 * y - z
func_2 = x - exp(y) + z
funcs = [func_1,func_2]

4) Run multi_func_all_derivatives()
result = multi_func_all_derivatives(funcs, in_vars, in_vals)

5) Print result.out_vals to get vector of calculated values of all funcs each at (in_vars)
print(result.out_vals)

6) Print result.out_derv_matrix to get the Hessian matrix of partial derivatives evaluated at (in_vars) print(result.out_derv_matrix)

**SOFTWARE ORGANIZATION**


At the moment, our directory structure is set up in such a way that our forward mode module is both easy to access and easy to test; the current structure is a temporary placeholder for a more comprehensive one to be implemented after we pin down some final specifics regarding our extension. We are still discussing whether or not to include the reverse mode of auto-differentiation in our package as the option-pricing extension would be viable without it; it would just be much slower than ideal if implemented with just the forward mode.
    
Either way, as it stands, the entirety of our forward mode implementation is contained in a single module, AutoDiff.py. Given functions of multiple variables and specific values, our module can evaluate both the functions and their partial derivatives at the given values. As of now, this single module is contained in a directory called “code.” Our test module - “test_AutoDiff.py” - also lives in this directory. The tests included in our test module are automatically run by TravisCI upon each commit to the remote repository. We haven’t experienced any issues with this testing method yet, so it seems that this is how we will test our code for the entirety of the project.
    
As for our directory structure moving forward, it is still dependent on whether we will be able to complete a fully functional module for the reverse mode of auto-differentiation. If we are, then the directory structure will be similar to the one outlined in our 1st milestone:
    
<img src="Directory_Structure.png" align="left"/>
    
If we are not able to implement the reverse mode, then we will have to edit this structure a little. Most likely, we will just have the option pricing module and the associated tests in our extension’s sub-package without the reverse mode implementation. 
    
We have not yet put our package on PyPI seeing as we still have a lot of structural elements to figure out. As of now, if a user wanted to use the package, he or she would have to download it independently, write the appropriate print statements at the bottom of AutoDiff.py (for the values they desired), and run the module from the command line.	

**IMPEMENTATION**



Our software will provide the user with a package he or she can use to evaluate functions of multiple variables at a specified point while at the same time evaluating the partial derivatives of the function at that same point. A class will be provided to perform simple calculations, and two functions will be provided to handle vector inputs and outputs.

Our strategy will be to create input-variable objects in python that once defined may be used in python calculations as if they were standard python variables of type float. However, when calculations are done with these Autdiff objects, a partial derivative with respect to one of the input variables will be calculated and carried along. The object representing the result, then, will contain an attribute that contains the value of the function at the specified point (val) and an attribute that contains a partial derivative with respect to one of the input variables (derv). Which partial derivative to calculate will be specified by the user by intiating the derv attribute of one input variable to 1 and the derv attributes of the other input variables to 0.  The calculation can then be repeated to get partial derivatives with respect to other input variables by changing which input variable’s derv attribute is intiated to 1.

*The AutDiff() class*

Most of the functionality of our package will be implemented in a single class called AutoDiff( ). Below is a skeleton of that class. As mentioned above, it will have two attributes self.val and self.derv that respectively will hold the value of the function being evaluated and the partial derivative of the function being evaluated up to some point in the calculation.

Autodiff() will have many methods. Each will be short and designed to replace a standard mathematical operation. Thus, for example there will be a method \_\_add\_\_ that will overload the standard addition operator (+) and thus specify how objects of type AutoDiff will be added to scalar values or to each other. And similar methods will be included to overload the other standard mathematical operators (- * / ^). Also, methods will be included to replace the 
numpy functions (exp, ln, log, sin, cos, tan, sqrt) so that these functions can be used as well in any calculations involving AutoDiff objects.

In the skeleton below, two of AutoDiff’s methods, multiplication and exponentiation, have been fully coded to illustrate how the desired functionality will be implemented generally. Central to all methods is the principle that each method will return a new AutDiff object that will represent the next stage in the function’s evaluation. For example, if x is an AutDiff object, and it is multiplied by the scalar 2 (x\*2), then x’s \_\_mul\_\_ method will be called and it will return a new object that will represent x\*2. The AutoDiff.\_\_mul\_\_ method will perform this calculation as follows. It will assume that 2 is another object of type AutoDiff, and it will try to multiply the two AutoDiff objects together. It will find that 2 is not an AutoDiff object, fall to its except block, and then make two calculations: 2 * x.val and 2 * x.derv and store these values in the returned object’s val and derv attributes. If the 2 had been to the right of x (2*x), then the \_\_rmul\_\_ method would have been called, and it would have executed the same operation.  

More subtle is the case where two AutoDiff objects, say x and y, are multiplied together x\*y. Here x’s \_\_mul\_\_ method will be called. It will find that the two objects are instances of AutoDiff, and it will 1) multiply x.val and y.val together and store the result as the returned object’s val attribute, and 2) calculate (x.derv\*y.val + y.derv\*x.val) and store this value as the returned object’s derv attribute. Notice in the second calculation the derivatives from both objects x and y are carried along as a sum. Recall, however, that the derv value of either x or y will have been initiated to 0, so one term in the sum (x.derv\*y.val + y.derv\*x.val) will always evaluate to 0. In this way, the algorithm can automatically choose the proper derivative to carry along for subsequent calculations (here either that of x or that of y) when two Autodiff objects collide. This same approach can be used more generally and will be used when overloading the division and power operators.

Also fully coded in the AutoDiff skeleton below is the exponential function. This works as follows. For a given AutoDiff object, say x, when exp(x) is executed, x’s exp() function will be called on itself. It will use numpy’s exp() function to evaluate np.exp(x.val) and store this value in the return object’s val attribute. It will evaluate the derivative of exp(x.value) —which in this case is just exp(x.value)— and store this value in the return object’s derv attribute. Our preliminary tests indicate that this will work for all the numpy fuctions: np.exp(), np.ln(), np.log(), np.sqrt(), np.sin(), np.cos(), np.tan(), so this is how they will be implemented. 

Autdiff objects will be fully functional replacements for python float variables such that once the AutoDiff class is defined, these objects can be used to perform forward autodifferentiation on any multi-input, single output function at any specified point. 

*Skeleton of AutoDiff()*

*The all_derivatives() function*

This function will allow the user to calculate all partial derivatives at once without having to manually initiate the derv attributes of the input variables several times and each time repeat the calculation.

Psuedo code for this function is listed below. It takes advantage of the AutoDiff() class.


    def all_derivatives(func, in_vars, in_vals):
        ***
        Inputs:

            func():   —the function to be evaluated
            in_vars:  —a vector containing the input variables.
            in_vals:  —a vector of values, one for each variable in in_vars that specify  
                         a point where the function and its partial derivatives are to be evaluated.
        Returns:

            func():    —the function evaluated
            val:       —the value of func( ) at the input point given by in_vals
            derv_vals: —a vector of partial derivative values, one for each input variable in in_vars.         	        
        ***

        Steps:
        * make a vector diff_vars which  coorepsonds to in_vars but all
          values are AutoDiff objects.
        * Set the derv value of the first element of diff_vars to 1 and the derv
          values of the rest of the diff_vars elements to 0.

        * Create the derv_vals vector that will hold the calculated partial derivatives

        * For i in range( the number of elements in in_vars):
            * Set the derv value of each element of in_vars to 0
            * Set  the derv value of the ith element to 1.
            * Evaluate func()  

            * If it is the first time thought the loop:
                  set val to the val of the AutoDiff output object
            * Else:
                * Store derivative of the output AutoDiff object as the ith element of derv_vals

        * Return  func, val, derv_vals


*The multi_func_all_derivatives() function*

This function will handle vector inputs and outputs. It will allow the user to specify: a vector of functions, a vector of input variables, and a vectors of values defining a specific point. It will return: the input vector containing the functions, a vector of output values (one for each function evaluated at the specified point), and a Hessian matrix of partial derivatives containing partial derivatives of all functions evaluated at the specified point. Consider the equation below as an example:

\begin{equation}
\begin{pmatrix} f(x,y) \\ g(x,y)\end{pmatrix} =
\begin{pmatrix} 2xy \\ x+y^2 \end{pmatrix} at \begin{pmatrix} x=a \\ y=b \end{pmatrix}
\end{equation}


Here the multi_func_all_derivatives() function would return:  
1) funcs, a vector of the functions that were evaluated ($2xy$,  $x+y^2$)  
2) out_vals, a vector of the values of each function after evaluation $(f(a,b), g(a,b))$  
3) out_derv_matrix, a matrix containing the partial derivatives evaluated at the point (a, b)

Pseudo code for this function is below. It takes advantge of the AutoDiff() class and the all_derivatives() function:

    def multi_func_all_derivatives(funcs, in_vars, in_vals):
        ***
        Inputs: 
            funcs:    —a vector of input functions
            in_vars: —a vector of input variables
            in_vals:  —a vector of input values specifying a point.

        Returns:
            funcs: 	   —the vector of input functions
            out_vals —a vector of output values, one for each function
            out_derv_matrix —the Hessian matrix of partial derivatives evaluated at the input point					 
        ***
        Steps:
        •	Create an empty vector out_vals
        •	Create an empty matrix out_derv_matrix
        •	For each function in funcs:
                - Run all_derivatives(func, in_vars, in_vals) 
                - Store the returned val in the vector out_vals
                - Store the returned vector derv_vals in the matrix out_derv_matrix

        •	Return funcs, out_vals, out_derv_matrix



**Future Features Proposal**

In the financial markets, large institutions (e.g. banks, asset managers) hold complex portfolios with a variety of financial instruments. While calculating the value of such portfolios can be done fairly easily with simulations and aggregation of market pricing, the sensitivities of these portfolios to changes in price, volatility and interest rates in the market can be a lot more difficult to calculate and come with an accuracy vs. efficiency trade off. 


Given the time and the scope of this project we chose to focus on a subset of the financial instruments and potential end users.  Specifically, we aim to extend our DeltaPi package into a second package HedgeDeltaPi that would be useful to options traders and as they estimate sensitivity of put and call option prices to changes in the price of the underlying asset. Accurate and efficient estimate of these sensitivities would allow options traders to buy the right amounts of underlying stocks to hedge their options positions from swings in the asset prices (more on this later).

Here we will focus on vanilla, European, non-dividend paying stock options that can only be exercised on specified date of their expiration as opposed to American options that can be exercised during any time up until the expiration date. 

***What are Financial Options?***

A call option is a right but not an obligation to buy an underlying asset (in our case stock) at a given price (strike price) on a given date (expiration date).  Note from the plot below that the option has no value below the strike price of USD100.  However, as the stock reaches that value and climbs higher, the value of the option is the difference between $100 and the current price (in the >100 region). That makes sense because we can buy the stock at a 100 and sell it at a price that is now >100, netting a profit that is now the value of the option.

$$\text{Value of European Call Option}$$

![](http://www.quantopia.net/wp-content/uploads/2013/03/EuroCallRates.png)</br>
$$\text{source: www.quantopia.net/wp-content/uploads/2013/03/EuroCallRates.png}$$

A put option is a right but not an obligation to sell an underlying asset (in our case stock) at a given price (strike) on a given date (expiration).  The plot below shows that the put option has no value until the stock price drops below the strike. Then the value of the option is the profit one would net by buying the stock <100 and selling it at a strike price of USD100. 

$$\text{Value of European Put Option}$$

![](http://www.quantopia.net/wp-content/uploads/2013/03/EuroPutRates.png)<br/>
$$\text{source: www.quantopia.net/wp-content/uploads/2013/03/EuroPutRates.png}$$

Black-Scholes Equation describes the option price overtime as follows:

$$\frac{\partial V}{\partial t}+\frac{1}{2}\sigma^2 S^2\frac{\partial^2 V}{\partial S^2}+r S\frac{\partial V}{\partial S} -r V = 0$$

where $V$ is the value of the option, $t$ is time, $\sigma$ is volatility of the underlying asset, $S$ is the price of the underlying asset and $r$ is the risk free rate (Black and Scholes, 1973).

Further, Black-Sholes-Merton formula for estimating price of the European Call option is as follows:

$$C(S_0,t)=S_0N(d_1)-Ke^{-r(T-t)}N(d_2)$$

and similarly the put option value is as follows:


$$P(S_0,t)=Xe^{-r(T-t)}N(-d_2)-S_tN(-d_1)$$

where $S_0$ is the stock price, $C(S_0,t)$ is a price of the call option and $P(S_0,t)$ is price of a put option as a function of stock price and time, $K$ is the strike prices, $(T-t)$ is time to maturity represented in years and $N(d_1)$ and $N(d_2)$ are  cumulative distribution functions for a standard normal distribution (Merton, 1973).

$$d_1=\frac{ln(\frac{S_t}{X})+(r+\frac{\sigma^2_s}{2})(T-t)}{\sigma_s\sqrt{(T-t)}}$$

$$d_2=\frac{ln(\frac{S_t}{X})+(r-\frac{\sigma^2_s}{2})(T-t)}{\sigma_s\sqrt{(T-t)}}=d_1-\sigma_s\sqrt{(T-t)}$$

$$N(d_1)=\int_{-\infty} ^{d_1}f(u)du=\int_{-\infty}^{d_1}\frac{1}{\sqrt{2\pi}}e^{\frac{-u^2}{2}}du$$

$$\dot{N(d_1)}=\frac{\partial N(d_1)}{\partial d_1}=\frac{1}{\sqrt{2\pi}}e^{-\frac{d_1^2}{2}}$$

In our extension package we will focus on the most widely used Greek - delta - for European put and call options. 

$$\Delta_{call}=\frac{\partial C_t}{\partial S_t}=N(d_1)+S_t\frac{\partial N(d_1)}{\partial S_t}-Xe^{-r(T-t)}\frac{\partial N(d_2)}{\partial S_t}$$

$$\Delta_{put}=\frac{\partial P_t}{\partial S_t}=Xe^{-r(T-t)}\frac{\partial N(-d_2)}{\partial S_t}-N(-d_1)-S_t\frac {\partial N(-d_1)}{\partial S_t}$$

***What is Delta Hedging?***

In the simple case of vanilla, non dividend paying, European call/put options above, delta hedging refers to buying or selling specific amounts of underlying stock to neutralize the impact of the stock price moves on the options portfolio.  Let us consider the following example:

Consider the following example. Let's supposed we have a stock XYZ that currently has a price of USD100 and the bank sold 10 call options (a while ago before the stock price run up) for 1000 shares with strike USD10.  Thus, the buyer of our option has the right but not obligation to buy 1000 shares of XYZ at just USD10 on expiration date (right now the option is deep in the money). If the delta of this call option is 0.5, this means that for every USD1 increase in stock price, the value of the call option goes up USD0.50 (to the holder).  To execute a delta hedge, the bank should buy 1000*.5=500 shares of XYZ. Now if the stock price goes down by USD1, the bank will lose USD500 on the stock position but will gain 0.5x1000=USD500 on its short call option position. Thus, the impact is neutral.  

Keep in mind that delta is not static as the slope of option price vs. stock price is usually non linear. This is why accurate and frequent estimates of delta are so important (Lee, 2008).

HedgeDeltaPi will apply automatic differentiation from DeltaPi to compute deltas for call and put options above. We will start by applying the forward mode and will also explore reverse mode AD as it is more efficient for scaling the packages to more complex portfolios (Homescu, 2011). 

**REFERENCES:**<br/>
Atilim Baydin. Automatic Differentiation in Machine Learning: a Survey. Journal of Machine Learning Research, 2018. <br/> 
Fischer Black, & Myron Scholes, M. The Pricing of Options and Corporate Liabilities. Retrieved November 1, 2019, from http://www.jstor.org/stable/1831029<br/>
Cheng-Few Lee, 'Handbook on Quantitative Finance and Risk Management.' Rutgers University. 2008.  
George F. Corliss. Application of differentiation arithmetic, volume 19 of Perspectives in Computing, pages 127–48. Academic Press, Boston, 1988.<br/>
Andreas Griewank and Andrea Walther. Evaluating Derivatives: Principles and Techniques of Algorithmic Differentiation. Humboldt-Universität zu Berlin, Berlin, Germany. Second Edition, 2008.<br/>
Michael Heath. Scientific Computing: An Introductory Survey. Society for Industrial and Applied Mathematics. 8(6):367, 2018.<br/> 
Christina Homescu. Adjoints and automatic (algorithmic) differentiation in computational finance. arXiv:1107.1831v1 10 Jul 2011.<br/>
Max E. Jerrell. Automatic differentiation and interval arithmetic for estimation of disequilibrium models. Computational Economics, 10(3):295–316, 1997.<br/>
Merton, R. Theory of Rational Option Pricing. Retrieved November 1, 2019, from http://www.jstor.org/stable/3003143 <br/>
David Sondak. Lectures 10 - 11. Harvard University. CS207 Fall 2019. <br/> 
